In [1]:
# for each site: 1 channel and 1 ground truth
# sort waves by height, pick test and train from each
# calc waves using CF calculated from each range
# Calculate waves from correction factor at each site

import numpy as np
import oceanDAS as odas
import h5py
import netCDF4 as nc
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, datetime, timezone
from scipy import signal

%matplotlib inline
%cd -q '/Users/hglover/Library/CloudStorage/Box-Box/FiberExperiments/DAScomparison_paper/'


In [4]:
pname = 'HomerAK_Williams/'
# load ground truth spectra:
with h5py.File(pname+'NDBC-46108_spectra.h5','r') as fp:
    sdate = fp['DATE'][:]/1e9 # POSIX time stamp (s) (convert from ns to s)
    sfreq = fp['FREQ'][:] # Hz
    sspec = fp['SPEC'][:] # m^2/Hz
nt = len(sdate)     # number of buoy spectra

# calculate wave stats from spectra for comparison:
hs_hombuoy = np.zeros(nt)
tp_hombuoy = np.zeros(nt)
te_hombuoy = np.zeros(nt)
for jj in range(nt):
    ds_psd_corr = sspec[jj,(sfreq > 0.04) & (sfreq < 0.3)]
    f_psd = sfreq[(sfreq > 0.04) & (sfreq < 0.3)]
    fe = ((ds_psd_corr * f_psd) / ds_psd_corr.sum() ).sum() #(f*E)/E
    te_hombuoy[jj] = 1/fe
    tp_hombuoy[jj] = 1/(f_psd[np.argmax(ds_psd_corr)])
    bandwidth = (f_psd[1::] - f_psd[0:-1]).mean()
    hs_hombuoy[jj] = 4*np.sqrt( ds_psd_corr.sum() * bandwidth )   
    

# load strain at all channels  then pull out channel 31
fname = 'GCI_TERRA_2p5Hz_data.h5'
onechn = False
rawstrain_all,das_time,chnl,depth,metadata = odas.loadHomer(pname,fname,onechn)
rawstrain_all = rawstrain_all[~np.isnan(das_time),:]
das_time = das_time[~np.isnan(das_time)]

ns,nx = rawstrain_all.shape

# Calculate strain spectra corresponding to each buoy spectra
nns = int(200*metadata['fs'])   # number of samples for each FFT (to match buoy resolution)
nsg = int(30*60*metadata['fs']) # number of samples in each window
chno = np.arange(nx)

5831980 65


In [5]:
rmse_hom_te = np.nan*np.empty(nx)
rmse_hom_tp = np.nan*np.empty(nx)
rmse_hom_hs = np.nan*np.empty(nx)

for chno in chno:
    print(chno)
    rawstrain = rawstrain_all[:,chno]
    # Calculate depth attenuation function to translate buoy surface to pressure at bed 
    L,_ = odas.dispersion(depth[chno],1 / sfreq)
    attenuation = np.exp((1/L)*np.abs(depth[chno])*(-1))**2 # square for energy
    attenuation[attenuation>150]=np.nan
    sspec = sspec*attenuation

    # calculate correction factor for each point in time
    ss = np.zeros((nt,len(sfreq)))
    for it in range(nt):
        ii = np.argmin(abs(sdate[it]-das_time))
        i1 = np.maximum(0,ii-nsg//2)
        i2 = np.minimum(ns,ii+nsg//2)
        freq_e,temp = signal.welch(rawstrain[i1:i2],fs=metadata['fs'],nperseg=nns,detrend=False,scaling='density')
        temp = np.interp(sfreq, freq_e, temp)
        ss[it,:] = sspec[it,:]/temp

    ss_med = np.nanmedian(ss,axis=0)

    hs_hom = np.zeros(len(sdate))*np.nan
    tp_hom = np.zeros(len(sdate))*np.nan
    te_hom = np.zeros(len(sdate))*np.nan
    for it in range(len(sdate)):
        ii = np.argmin(abs(sdate[it]-das_time))
        i1 = np.maximum(0,ii-nsg//2)
        i2 = np.minimum(ns,ii+nsg//2)
        tp_hom[it], te_hom[it], hs_hom[it] = odas.DAS_wave_conversion(
            rawstrain[i1:i2],metadata['fs'],depth[chno],ss_med,sfreq,0.27)
    tvec_hom = sdate

    # calculate r2 and mse
    mask = np.isfinite(te_hom) & np.isfinite(te_hombuoy)
    rmse_hom_te[chno] = np.sqrt(np.mean((te_hom[mask]-te_hombuoy[mask])**2))
    mask = np.isfinite(hs_hom) & np.isfinite(hs_hombuoy)
    rmse_hom_hs[chno] = np.sqrt(np.mean((hs_hom[mask]-hs_hombuoy[mask])**2))
    mask = np.isfinite(tp_hom) & np.isfinite(tp_hombuoy)
    rmse_hom_tp[chno] = np.sqrt(np.mean((tp_hom[mask]-tp_hombuoy[mask])**2))

# calculate distance from buoy for each channel: https://edwilliams.org/avform147.htm

blon = -151.8291
blat = 59.5975

blon *= np.pi/180
blat *= np.pi/180

fname = 'GCI_TERRA_2p5Hz_data.h5'

with h5py.File(fname,'r') as fp:
    lon2 = fp['Longitude'][:]*np.pi/180
    lat2 = fp['Latitude'][:]*np.pi/180
# print(lon,lat)

d=2*np.arcsin(np.sqrt((np.sin((blat-lat2)/2))**2 + np.cos(blat)*np.cos(lat2)*(np.sin((blon-lon2)/2))**2))
d *=180*60/np.pi*1852
np.savez(pname+'waveStatsHomer_allCh',rmse_te=rmse_hom_te,rmse_tp=rmse_hom_tp,rmse_hs=rmse_hom_hs,dist=d)

0


/Users/hglover/Library/CloudStorage/Box-Box/HannahGlover/Research/code/compare_das/oceanDAS.py:324: RuntimeWarning: divide by zero encountered in true_divide
  L,_ = dispersion(depth,1/f_psd)
/Users/hglover/Library/CloudStorage/Box-Box/HannahGlover/Research/code/compare_das/oceanDAS.py:290: RuntimeWarning: divide by zero encountered in power
  k = omega ** 2 / g * (1 - np.exp(-(omega * np.sqrt(h / g)) ** (5/2))) ** (-2/5)
/Users/hglover/Library/CloudStorage/Box-Box/HannahGlover/Research/code/compare_das/oceanDAS.py:290: RuntimeWarning: invalid value encountered in multiply
  k = omega ** 2 / g * (1 - np.exp(-(omega * np.sqrt(h / g)) ** (5/2))) ** (-2/5)
/Users/hglover/Library/CloudStorage/Box-Box/HannahGlover/Research/code/compare_das/oceanDAS.py:333: RuntimeWarning: divide by zero encountered in power
  ds_psd_corr[np.isnan(ds_psd_corr)] = (min_psd)*np.power(f_noise[0],4)*np.power(f_noise,-4)
/Users/hglover/Library/CloudStorage/Box-Box/HannahGlover/Research/code/compare_das/oceanDAS.p

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'GCI_TERRA_2p5Hz_data.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# plot calculated wave chars for Oliktok
pname = 'OliktokPoint_Smith/'

d = nc.Dataset(pname+'CODAS_2022-08_DASwaveoutputs_v230612.nc')
time_oli = d['time'][:]
hs_oli = d['Hs'][:]
tp_oli = d['Tp'][:]
te_oli = d['Te'][:]

nx,_ = te_oli.shape
chno = np.arange(nx)
d = nc.Dataset(pname+'SWIFT18_OliktokPt_Aug2022_reprocessedIMU_1Hzspectra_convertedTe.nc')
time_swift = d['time'][:]
hs_swift = d['sigwaveheight_new'][:]
tp_swift = d['peakwaveperiod'][:]
te_swift = d['energywaveperiod'][:]

hs_swift = np.interp(time_oli,time_swift,hs_swift)
tp_swift = np.interp(time_oli,time_swift,tp_swift)
te_swift = np.interp(time_oli,time_swift,te_swift)



rmse_oli_te = np.nan*np.empty(nx)
rmse_oli_tp = np.nan*np.empty(nx)
rmse_oli_hs = np.nan*np.empty(nx)

for chno in chno:
    # calculate r2 and mse
    mask = np.isfinite(te_oli[chno,:]) & np.isfinite(te_swift)
    rmse_oli_te[chno] = np.sqrt(np.mean((te_oli[chno,mask]-te_swift[mask])**2))
    mask = np.isfinite(tp_oli[chno,:]) & np.isfinite(tp_swift)
    rmse_oli_tp[chno] = np.sqrt(np.mean((tp_oli[chno,mask]-tp_swift[mask])**2))
    mask = np.isfinite(hs_oli[chno,:]) & np.isfinite(hs_swift)
    rmse_oli_hs[chno] = np.sqrt(np.mean((hs_oli[chno,mask]-hs_swift[mask])**2))

    
    
np.savez(pname+'waveStatsOliktok_allCh',rmse_te=rmse_oli_te,rmse_tp=rmse_oli_tp,rmse_hs=rmse_oli_hs)